In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [8]:
# Caregando o arquivo
df = pd.read_csv('./house-prices-advanced-regression-techniques/train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [9]:
# Selecionando os atributos para a exploração
df.shape[0]
df['LotFrontage'][0]

mydf = df[['LotArea','Street','BldgType','HouseStyle','OverallQual', 'OverallCond','YearBuilt','TotalBsmtSF','BedroomAbvGr','KitchenAbvGr','SalePrice']]
mydf.head()

,LotArea,Street,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,TotalBsmtSF,BedroomAbvGr,KitchenAbvGr,SalePrice
0,8450,Pave,1Fam,2Story,7,5,2003,856,3,1,208500
1,9600,Pave,1Fam,1Story,6,8,1976,1262,3,1,181500
2,11250,Pave,1Fam,2Story,7,5,2001,920,3,1,223500
3,9550,Pave,1Fam,2Story,7,5,1915,756,3,1,140000
4,14260,Pave,1Fam,2Story,8,5,2000,1145,4,1,250000


In [10]:
# Descrição do conjunto de dados selecionado, nome: mydf
mydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 11 columns):
LotArea         1460 non-null int64
Street          1460 non-null object
BldgType        1460 non-null object
HouseStyle      1460 non-null object
OverallQual     1460 non-null int64
OverallCond     1460 non-null int64
YearBuilt       1460 non-null int64
TotalBsmtSF     1460 non-null int64
BedroomAbvGr    1460 non-null int64
KitchenAbvGr    1460 non-null int64
SalePrice       1460 non-null int64
dtypes: int64(8), object(3)
memory usage: 125.5+ KB


In [11]:
# Fazendo a conversão dos atributos qualitativos 
print(mydf['Street'].value_counts())
print(mydf['BldgType'].value_counts())
print(mydf['HouseStyle'].value_counts())
cleanup_nums = {"Street": {"Grvl":0, "Pave":1},
                 "BldgType": {"1Fam":0, "TwnhsE": 1, "Duplex":2, "Twnhs":3, "2fmCon":4},
                 "HouseStyle": {"1Story":0, "2Story":2, "1.5Fin":3, "SLvl":4, "SFoyer":5, "1.5Unf":6, "2.5Unf":7, "2.5Fin":8}}
mydf.replace(cleanup_nums, inplace=True)
print(mydf['Street'].value_counts())
print(mydf['BldgType'].value_counts())
print(mydf['HouseStyle'].value_counts())

Pave    1454
Grvl       6
Name: Street, dtype: int64
1Fam      1220
TwnhsE     114
Duplex      52
Twnhs       43
2fmCon      31
Name: BldgType, dtype: int64
1Story    726
2Story    445
1.5Fin    154
SLvl       65
SFoyer     37
1.5Unf     14
2.5Unf     11
2.5Fin      8
Name: HouseStyle, dtype: int64
1    1454
0       6
Name: Street, dtype: int64
0    1220
1     114
2      52
3      43
4      31
Name: BldgType, dtype: int64
0    726
2    445
3    154
4     65
5     37
6     14
7     11
8      8
Name: HouseStyle, dtype: int64


In [13]:
mydf.describe()

,LotArea,Street,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,TotalBsmtSF,BedroomAbvGr,KitchenAbvGr,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,0.995890,0.322603,1.384932,6.099315,5.575342,1971.267808,1057.429452,2.866438,1.046575,180921.195890
std,9981.264932,0.063996,0.849567,1.631944,1.382997,1.112799,30.202904,438.705324,0.815778,0.220338,79442.502883
min,1300.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1872.000000,0.000000,0.000000,0.000000,34900.000000
25%,7553.500000,1.000000,0.000000,0.000000,5.000000,5.000000,1954.000000,795.750000,2.000000,1.000000,129975.000000
50%,9478.500000,1.000000,0.000000,2.000000,6.000000,5.000000,1973.000000,991.500000,3.000000,1.000000,163000.000000
75%,11601.500000,1.000000,0.000000,2.000000,7.000000,6.000000,2000.000000,1298.250000,3.000000,1.000000,214000.000000
max,215245.000000,1.000000,4.000000,8.000000,10.000000,9.000000,2010.000000,6110.000000,8.000000,3.000000,755000.000000


In [15]:
import os
files = [f for f in os.listdir('./') if os.path.isfile(f)]
# print(files)

In [16]:
mydf

,LotArea,Street,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,TotalBsmtSF,BedroomAbvGr,KitchenAbvGr,SalePrice
0,8450,1,0,2,7,5,2003,856,3,1,208500
1,9600,1,0,0,6,8,1976,1262,3,1,181500
2,11250,1,0,2,7,5,2001,920,3,1,223500
3,9550,1,0,2,7,5,1915,756,3,1,140000
4,14260,1,0,2,8,5,2000,1145,4,1,250000
5,14115,1,0,3,5,5,1993,796,1,1,143000
6,10084,1,0,0,8,5,2004,1686,3,1,307000
7,10382,1,0,2,7,6,1973,1107,3,1,200000
8,6120,1,0,3,7,5,1931,952,2,2,129900
9,7420,1,4,6,5,6,1939,991,2,2,118000


In [41]:
####PROBARRRR
from sklearn.neural_network import MLPRegressor
import random

size_col = 5
N_test = 10
size_subset = int(mydf.shape[0]/10)
print('size subset ' + str(size_subset))
x_subset = np.zeros([size_subset,size_col+1])

mydf_data = mydf.get_values()

vec_row = np.zeros([ N_test, size_subset])
vec_col = np.zeros([ N_test, size_col ])
vec_diff = np.zeros([ 1, N_test ])
# print(vec_diff.shape)

for n_test in range(0,N_test):
    print( n_test, end = ' ' )
    x_row = random.sample(range(1, mydf.shape[0]), size_subset)
    x_row = np.sort(x_row)
#     print(x_row)
    vec_row[n_test,:] = x_row

#     print( mydf.shape )
    x_col = random.sample(range(0, mydf_data.shape[1]-1), size_col)
    x_col = np.sort(x_col)
#     print(x_col)
    vec_col[n_test,:] = x_col
    
    c = 0
    
    for idj in range(0, mydf_data.shape[1]-1):
#         print('idj ' + str(idj) + ' ' + str(x_col[c]))
        for idx_subset in range(0,size_subset):
            if( idj == x_col[c] ):
                x_subset[idx_subset,c] = mydf_data[x_row[idx_subset],x_col[c]]
#         print(c)
        if(idj == x_col[c]):
            c += 1
        if( c == size_col ):
            break;
        
    
    for idx_subset in range(0,size_subset):
        x_subset[idx_subset,x_subset.shape[1]-1] = mydf_data[x_row[idx_subset],mydf_data.shape[1]-1]

#     print(x_subset)

    x_st = x_subset[:,0:x_subset.shape[1]-1]
    y_st = x_subset[:,x_subset.shape[1]-1]
#     print(x_subset[:,0:x_subset.shape[1]-1])
#     print(x_subset[:,x_subset.shape[1]-1])
        
    clf = MLPRegressor(hidden_layer_sizes=(100, ), activation='relu', solver='adam', alpha=0.0001).fit(x_st,y_st)
    # print(clf.predict(x_t))
    
    y_pdt = clf.predict(x_st)
    counter = 0
    for i in range(0,x_st.shape[0]):
        counter += (y_st[i] - y_pdt[i])*(y_st[i] - y_pdt[i])
    
    vec_diff[0,n_test] = counter
    print(str(clf.score(x_st,y_st))+ ' ' + str(counter))

size subset 146
0 -2.9404456048747787 4648349082758.921
1 -5.959406742426315 4564037114583.589
2 -4.484551950391188 5213804389668.118
3 -3.247439711367484 3962107133869.901
4 -1.7634906756967128 4396942645315.9
5 -5.692596650048156 5026056015337.655
6 -2.6537766724783207 3612299900660.6445
7 -3.442393992424715 3722580090744.942
8 -4.612011992186872 6539776145249.167
9 -5.155446451569579 5352950117967.906


In [51]:
my_min = vec_diff[0,0]
idx = 0
for i in range(1,N_test):
    if( my_min > vec_diff[0,i]):
        my_min = vec_diff[0,i]
        idx = i
print(idx)
print(np.min(vec_diff[0,:]))
print(vec_row[idx,:])
print(vec_col[idx,:])

6
3612299900660.6445
[  10.   17.   24.   26.   27.   33.   37.   61.   71.   75.   84.   91.
   93.  107.  108.  110.  124.  166.  171.  173.  201.  203.  209.  217.
  218.  219.  252.  255.  268.  277.  282.  285.  286.  287.  289.  292.
  296.  320.  321.  325.  358.  363.  381.  382.  385.  389.  391.  410.
  412.  422.  426.  427.  444.  452.  462.  463.  510.  525.  529.  544.
  561.  577.  583.  585.  599.  613.  626.  631.  648.  664.  711.  720.
  762.  779.  795.  796.  799.  801.  811.  828.  833.  851.  853.  857.
  871.  880.  888.  891.  897.  898.  913.  931.  942.  950.  962.  963.
  971.  973.  993. 1002. 1027. 1028. 1040. 1044. 1058. 1061. 1063. 1077.
 1078. 1099. 1112. 1114. 1115. 1135. 1140. 1150. 1162. 1164. 1189. 1205.
 1216. 1218. 1219. 1232. 1255. 1257. 1260. 1283. 1300. 1332. 1343. 1368.
 1373. 1391. 1392. 1405. 1414. 1426. 1429. 1433. 1437. 1440. 1447. 1450.
 1458. 1459.]
[0. 1. 4. 6. 9.]
